In [ ]:
!pip install \
  transformers==4.57.3 \
  bitsandbytes==0.49.0 \
  accelerate==1.12.0 \
  sentence-transformers==5.1.2 \
  faiss-cpu==1.13.1 \
  datasets==4.0.0 \
  evaluate==0.4.6

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from accelerate import init_empty_weights
from transformers import BitsAndBytesConfig

device = "cuda" # the device to load the model onto

# Initialize the model with quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Set to True for 4-bit quantization or False for 8-bit
    bnb_4bit_use_double_quant=True,  # Optional: Improves stability in 4-bit quantization
    bnb_4bit_quant_type="nf4",  # Optional: Use 'nf4' for better accuracy or 'fp4' for faster computation
    bnb_4bit_compute_dtype=torch.float16  # Optional: use float16 for better performance on newer GPUs
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config  # Pass the quantization configuration
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

## Classification with Large Language Models

So far, we have taken a look at using LLMs for tasks such as *open-ended question answering* and *machine translation* - both are what we call **generative tasks**. In simplified terms this means that we expect the model to generate outputs that may be very different depending on the input context we provide. The answer to *"Give me a short introduction to large language models."* will be structurally and semantically very different to *the German translation of an input sentence*.

Another type of tasks we can tackle with large language models are **discriminative (classification) tasks**.  
Given an input text, the goal in classification tasks is to assign it one* out of a small set of possible labels. One example of a classification task we will take a look at is called **sentiment classification**.

\* For simplicity, we are only dealing with single-label classification.

In sentiment classification, the goal is to identify the emotional polarity expressed in a text. Usually, a text may have a **positive**, **negative** or **neutral** sentiment.

The way we are going to approach the discriminative task is going to be very similar to the previous (generative) tasks, reusing much of the code from the notebook *llm-machine-translation*. **Very importantly however, the answer we expect from the model is significantly less free-form than before**. To this extent, we are going to specify the valid answer options for the LLM in its prompt, asking it to respond with either "Positive", "Negative", or "Neutral".

In [ ]:
prompt = ("Classify the sentiment of the following text as 'Positive', 'Negative', or 'Neutral'.\n\n"
          "Today is a good day.")
messages = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.6,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Constraining the output structure through the prompt already works quite well, guiding the model to answer with one of the valid answers.
By providing constraints in the prompt, we are attempting to limit the possible next words generated by the LLM to only *Positive*, *Negative*, and *Neutral*. In technical terms, we are trying to achieve probability zero for all other words.

In practice, however, guiding the model through the prompt alone is not a robust way to limit the valid answers. Numerous next words maintain a small yet nonzero probability, meaning the LLM may occasionally answer with additional options.
For example, instead of generating a single-word response 
#### *Positive* 
the model might output a full sentence response 
#### *The sentiment of the given text is Positive*.

# Guided decoding
A more robust way to constrain the set of possible answers is by using **guided decoding**. Broadly speaking, [guided decoding](https://huggingface.co/blog/nmmursit/guided-decoding) is a set of methods for enforcing that the model response follows a structure. Simplified for our purpose (classification), the idea of guided decoding is to set the probability of all invalid (unspecified) answers to 0. This means that only the valid answers may be generated by the LLM.

To demonstrate this in practice, we will use **vLLM**, a convenient library for working with LLMs efficiently. We take a more detailed look at the library in the notebook *faster-inference-vllm.ipynb*. 

In [ ]:
# the vLLM library uses its own methods for working with the models, so we have to reload the model
del model
!pip install vllm==0.10.2 bitsandbytes==0.46.1 transformers==4.57.6

In [ ]:
from vllm import LLM, SamplingParams
from vllm.sampling_params import GuidedDecodingParams
import torch
import os
os.environ["VLLM_USE_V1"] = "0"

model = LLM(
    "Qwen/Qwen2-1.5B-Instruct",
    dtype=torch.float16,
    trust_remote_code=True,
    quantization="bitsandbytes",
    gpu_memory_utilization=0.8,
    max_model_len=2048
)

In [ ]:
guided_decoding_params = GuidedDecodingParams(choice=["Positive", "Negative", "Neutral"])
sampling_params = SamplingParams(guided_decoding=guided_decoding_params)
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
)

response = outputs[0].outputs[0].text
print(response)

ERROR 01-01 13:19:14 [core_client.py:564] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.


## Exercises
1. How well does the model work using the simple prompt? Gather a small sample of positive, negative, and neutral texts. Classify the texts with a LLM, and compare the predicted answers to the correct ones.
2. Try to improve the classification accuracy by modifying the prompt. Two things you can try doing is:  
- including a more detailed definition of the sentiment classification task;
- including demonstrations of input texts and their sentiment.